In [9]:
import paho.mqtt.client as mqtt
import time
def on_log(client, userdata, level, buf):
    print("log: ",buf)

def on_connect(client, userdata, flags, rc):
    if rc==0:
        print("Connected OK")
    else:
        print("Bad connection returned code=",rc)
def on_disconnect(client, userdata, flags, rc=0):
    print("Disconnected result code "+str(rc))
def on_message(client, userdata, msg):
    topic=msg.topic
    m_decode=str(msg.payload.decode("utf-8","ignore"))
    print("message recieved", m_decode)


broker = "test.mosquitto.org"
client = mqtt.Client("tracing1")
counter = 0

client.on_connect=on_connect
client.on_disconnect=on_disconnect
# client.on_log=on_log
client.on_message=on_message

print("Connecting to broker: ", broker)
client.connect(broker)
client.loop_start()
client.subscribe("tracing")
client.publish("tracing","Hallo Server")
while(counter<1000):
    time.sleep(4)
    counter += 1
client.loop_stop()

client.disconnect()


Connecting to broker:  test.mosquitto.org
log:  Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b'tracing1'
log:  Sending SUBSCRIBE (d0, m1) [(b'tracing', 0)]
log:  Sending PUBLISH (d0, q0, r0, m2), 'b'tracing'', ... (12 bytes)
log:  Received CONNACK (0, 0)
Connected OK
log:  Received SUBACK
log:  Received PUBLISH (d0, q0, r0, m0), 'tracing', ...  (12 bytes)
message recieved Hallo Server
log:  Sending DISCONNECT
Disconnected result code 0


0